In [ ]:
from astrometry.util.fits import *
%matplotlib inline
import pylab as plt
import numpy as np
import json
from astrometry.util.util import Tan, Sip, fit_sip_wcs_py
from astrometry.util.starutil_numpy import radectoxyz, arcsec_between
from scipy.interpolate import InterpolatedUnivariateSpline
import fitsio

In [ ]:
from mappings import *

guide_loc = 8
petal_id = petal_loc_to_id[guide_loc]
gfa_num = petal_id_to_gfa_num[petal_id]
print('Petal id', petal_id)
print('GFA#', gfa_num)

fn = 'data/sip-petal%i.fits' % petal_id
print('Reading', fn)
labwcs = Sip(fn)
hdr = fitsio.read_header(fn)

In [ ]:
petal_loc_to_id

In [ ]:
prefix = '32840-10-GUIDE%i' % guide_loc
skywcs = Tan(prefix + '.wcs')
xy = fits_table(prefix + '.axy')
rd = fits_table(prefix + '.rdls')
img = fitsio.read(prefix + '.fits')
corr = fits_table(prefix + '.corr')

In [ ]:
print(skywcs)

In [ ]:
skycd = np.array(skywcs.cd).reshape((2,2))
#thsky = np.arctan2(skycd[0,1], skycd[0,0])
# x axis
#thsky = np.arctan2(skycd[1,0], skycd[0,0])
thsky = np.arctan2(skycd[1,0]+skycd[1,1], skycd[0,0]+skycd[0,1])


In [ ]:
thsky

In [ ]:
labcd = np.array(labwcs.wcstan.cd).reshape((2,2))
#thlab = np.arctan2(labcd[0,1], labcd[0,0])
# x axis
#thlab = np.arctan2(labcd[1,0], labcd[0,0])
# avg of x,y axes
thlab = np.arctan2(labcd[1,0]+labcd[1,1], labcd[0,0]+labcd[0,1])


In [ ]:
thlab

In [ ]:
labcd

In [ ]:
skycd

In [ ]:
dth = thsky - thlab
R = np.array([[np.cos(dth), -np.sin(dth)],[np.sin(dth), np.cos(dth)]])
newcd = np.dot(R, labcd)

In [ ]:
newcd

In [ ]:
np.rad2deg(dth)

In [ ]:
plt.plot([0, skycd[0,0]], [0, skycd[1,0]], 'b-')
plt.plot([0, skycd[0,1]], [0, skycd[1,1]], 'c-')

plt.plot([0, newcd[0,0]], [0, newcd[1,0]], 'r-')
plt.plot([0, newcd[0,1]], [0, newcd[1,1]], 'm-')

plt.plot([0, labcd[0,0]], [0, labcd[1,0]], 'g-')
plt.plot([0, labcd[0,1]], [0, labcd[1,1]], 'k-')

plt.axis('equal');

In [ ]:
fitwcs = Sip(labwcs)

In [ ]:
fitwcs.wcstan.set_cd(*newcd.ravel())
fitwcs.wcstan.set_crval(*skywcs.crval)

In [ ]:
print(fitwcs)

In [ ]:
plt.figure(figsize=(12,6))
#refra = rd.ra
#refdec = rd.dec
refra = corr.index_ra
refdec = corr.index_dec
ok,tx,ty = fitwcs.radec2pixelxy(refra, refdec)
mn,mx = np.percentile(img.ravel(), [50,99])
plt.imshow(np.minimum(img,mx), interpolation='nearest', origin='lower', vmin=mn, vmax=mx*1.2, cmap='gray');
ax = plt.axis()
plt.plot(tx-1, ty-1, 'o', mec='r', mfc='none',ms=10, mew=2)

imx = corr.field_x/1.1
imy = corr.field_y
plt.plot(imx-1, imy-1, '+', mec='c', mfc='none', ms=15, mew=2);
#plt.axis(ax)

In [ ]:
# Undo SIP distortion for pixel locations of stars
# Re-fit to reference stars for the TAN terms (with CRPIX=center)

In [ ]:
# SIP_pixelxy2radec: sip_distortion -> tan_pixelxy2radec
# xy2radec: xy2iwc, iwc2xyz, xyz2rd

# Re-fit: CRVAL, CD rotation.  Scale?

In [ ]:
dixy = np.array([fitwcs.get_distortion(xi,yi) for xi,yi in zip(imx, imy)])

In [ ]:
dix = dixy[:,0]
diy = dixy[:,1]

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(np.minimum(img,mx), interpolation='nearest', origin='lower', vmin=mn, vmax=mx*1.2, cmap='gray');
ax = plt.axis()
plt.plot(imx-1, imy-1, '+', mec='r', mfc='none', ms=15, mew=2);
plt.plot(dix-1, diy-1, '+', mec='c', mfc='none', ms=15, mew=2);

In [ ]:
fittan = Tan(fitwcs.wcstan)
def move_tan_1(intan, dr, dd, rot):
    otan = Tan(intan)
    cra,cdec = otan.crval
    cd = np.array(otan.cd).reshape((2,2))
    otan.set_crval(*(cra+dr, cdec+dd))
    R = np.array([[np.cos(rot), -np.sin(rot)],[np.sin(rot), np.cos(rot)]])
    rcd = np.dot(R, cd)
    otan.set_cd(*rcd.ravel())
    return otan
    
def objective_1(params):
    dr,dd,rot = params
    otan = move_tan_1(fittan, dr, dd, rot)
    ok,xx,yy = otan.radec2pixelxy(refra, refdec)
    return np.sum(np.hypot(xx - dix, yy - diy))

In [ ]:
def move_tan_2(intan, dr, dd, rot, scale):
    otan = Tan(intan)
    cra,cdec = otan.crval
    cd = np.array(otan.cd).reshape((2,2))
    otan.set_crval(*(cra+dr, cdec+dd))
    R = np.array([[np.cos(rot), -np.sin(rot)],[np.sin(rot), np.cos(rot)]])
    rcd = np.dot(R, cd)
    otan.set_cd(*((1.+scale) * rcd.ravel()))
    return otan
    
def objective_2(params):
    dr,dd,rot, scale = params
    otan = move_tan_2(fittan, dr, dd, rot, scale)
    ok,xx,yy = otan.radec2pixelxy(refra, refdec)
    return np.sum(np.hypot(xx - dix, yy - diy))

In [ ]:
from scipy.optimize import minimize

In [ ]:
res1 = minimize(objective_1, np.array([0,0,0]))
res1

In [ ]:
res2 = minimize(objective_2, np.array([0.,0.,0.,0.]))
res2

In [ ]:
opttan = move_tan_1(fittan, *res1.x)
optsip = Sip(fitwcs)
optsip.wcstan = opttan

In [ ]:
opttan2 = move_tan_2(fittan, *res2.x)
optsip2 = Sip(fitwcs)
optsip2.wcstan = opttan2

In [ ]:
print(fittan)
print(opttan)
print(optsip)
print(optsip2)

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(np.minimum(img,mx), interpolation='nearest', origin='lower', vmin=mn, vmax=mx*1.2, cmap='gray');
ax = plt.axis()
plt.plot(imx-1, imy-1, '+', mec='c', mfc='none', ms=15, mew=2);
ok,tx,ty = optsip.radec2pixelxy(refra, refdec)
plt.plot(tx-1, ty-1, 'o', mec='r', mfc='none',ms=10, mew=2);

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(np.minimum(img,mx), interpolation='nearest', origin='lower', vmin=mn, vmax=mx*1.2, cmap='gray');
ax = plt.axis()
plt.plot(imx-1, imy-1, '+', mec='c', mfc='none', ms=15, mew=2);
ok,tx,ty = optsip2.radec2pixelxy(refra, refdec)
plt.plot(tx-1, ty-1, 'o', mec='r', mfc='none',ms=10, mew=2);

In [ ]:
gif1xy = np.array([(hdr['GIF1X%i'%i], hdr['GIF1Y%i'%i]) for i in range(1,5)])
gif2xy = np.array([(hdr['GIF2X%i'%i], hdr['GIF2Y%i'%i]) for i in range(1,5)])

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(np.minimum(img,mx), interpolation='nearest', origin='lower', vmin=mn, vmax=mx*1.2, cmap='gray');
ax = plt.axis()
plt.plot(imx-1, imy-1, '+', mec='c', mfc='none', ms=15, mew=2);
ok,tx,ty = optsip2.radec2pixelxy(refra, refdec)
plt.plot(tx-1, ty-1, 'o', mec='r', mfc='none',ms=10, mew=2);
plt.plot(gif1xy[:,0], gif1xy[:,1], 'r.')
plt.plot(gif2xy[:,0], gif2xy[:,1], 'b.');

In [ ]:
gif1ra,gif1dec = optsip.pixelxy2radec(gif1xy[:,0], gif1xy[:,1])
gif2ra,gif2dec = optsip.pixelxy2radec(gif2xy[:,0], gif2xy[:,1])

In [ ]:
h,w = 1032,2048
ccdbx,ccdby = [1,w,w,1,1], [1,1,h,h,1]
ccdra,ccddec = optsip.pixelxy2radec(ccdbx, ccdby)

#sra,sdec = skywcs.pixelxy2radec(ccdbx, ccdby)
#plt.plot(sra, sdec, 'g-');
#plt.plot(sra[0], sdec[0], 'go');

plt.plot(ccdra, ccddec, 'k-');
plt.plot(ccdra[0], ccddec[0], 'ko');
plt.plot(refra, refdec, 'b+');
plt.plot(gif1ra, gif1dec, 'r.')
plt.plot(gif2ra, gif2dec, 'b.')
plt.axis('equal')
xl,xh = plt.xlim()
plt.xlim(xh,xl);

In [ ]:
for g in [0,2,3,5,7]:#,8]:
    fn = 'gfa-28205-GUIDE%i.wcs' % g
    wcs = Tan(fn)
    ra,dec = wcs.pixelxy2radec(ccdbx, ccdby)
    plt.plot(ra, dec, 'k-')
    plt.plot(ra[0], dec[0], 'ko')
    plt.text(np.mean(ra), np.mean(dec), 'GUIDE%i'%g)
xl,xh = plt.xlim()
plt.xlim(xh,xl)
plt.xlabel('RA (deg)')
plt.ylabel('Dec (deg)')
